In [2]:
import cv2
import torch
from ultralytics import YOLO
from sort import *

In [29]:
cap = cv2.VideoCapture(r"C:\Users\aashutosh kumar\Downloads\CARS.mp4")
model = YOLO("yolov8l.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

mask = cv2.imread(r"C:\Projects\Car-Counter\CAR-COUNTER\MASK1.png")
mask = cv2.resize(mask, (640, 360))  # Resize mask to match the frame size
limits = [100, 250, 600, 250]
while True:
    r, frame = cap.read()

    if r == True:
        frame_Rgn = cv2.bitwise_and(frame, mask)
        results = model(frame_Rgn, stream=True)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                class_id = int(box.cls[0])
                confidence = box.conf[0]
                class_name = classNames[class_id]
                if class_name == "car":
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (150, 200, 0), 3, 1)
                    label = f"{class_name} {confidence:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 200, 0), 1)
        cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
        cv2.imshow("ROI", frame_Rgn)
        cv2.imshow("Cars", frame)

        if cv2.waitKey(25) & 0xff == ord("p"):
            break

cv2.destroyAllWindows()
cap.release()



0: 384x640 5 cars, 1 bus, 33.1ms
Speed: 2.1ms preprocess, 33.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.8ms
Speed: 2.1ms preprocess, 30.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 29.8ms
Speed: 1.1ms preprocess, 29.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 31.3ms
Speed: 1.0ms preprocess, 31.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.2ms
Speed: 2.5ms preprocess, 30.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.3ms
Speed: 2.1ms preprocess, 30.3ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 30.6ms
Speed: 2.0ms preprocess, 30.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 29.2ms
Speed: 1.0ms preprocess, 29.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)



In [24]:
mask.shape

(360, 640, 3)